tempo:
- ammazzare 

- ingannare 

- pioggia 

- mele 

- spazio

Cercare le parole nelle frasi del dataset, restiruire le frasi che le cotnengono.
pulire le frasi ottenute, per restituire "tempo" da "ammazzare il tempo" (stopwords e initialwords)
restituirer la parola più frequente tra quelle restituite

In [207]:
from nltk.corpus import stopwords
import urllib.parse as up
import urllib.request as urlo
import json
import gzip
from io import BytesIO
#print(stopwords.words('italian'))

In [208]:
WORDS_1 = ['ammazzare', 'ingannare', 'pioggia', 'mele', 'spazio']
WORDS_2 = ['bicchiere', 'mulino', 'gocce', 'gola', 'buco']
WORDS_3 = ['campa', 'battaglia', 'razza', 'Troia', 'vecchio']
WORDS_4 = ['giallo', 'cane', 'scimmia', 'rubato', 'amore']
WORDS_5 = ['attività', 'famiglia', 'rubato', 'fiume', 'nido']
WORDS_6 = ['color', 'dietro', 'pietra', 'bianca', 'andare']
WORDS_7 = ['bollicine', 'da Gama']

In [209]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

In [210]:
def check_dataset(path, words):
    phrases = []    
    with open(path, 'r') as file:
        for line in file:
            for word in words:
                if word.lower() in line.lower():  # Salviamo le frasi che contengono le parole in input
                    line = line.lower()
                    line = line.replace(word, '')
                    phrases.append(remove_stop_words_ita(line)) # Rimuoviamo dalla frase le stopword e le parole in input
    return phrases

In [211]:
def most_frequent_words(phrases):
    words = {}
    for phrase in phrases:
        for word in phrase:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    return words

In [212]:
def get_most_frequent_word(words):
    max_word = ''
    max_count = 0
    for word in words:
        if words[word] > max_count:
            max_word = word
            max_count = words[word]
    return max_word

In [213]:
ph = check_dataset('..\ghigliottina.txt', WORDS_7)
#print(ph)
dict = most_frequent_words(ph)
print(dict)
answer = get_most_frequent_word(dict)
print(f"\n{WORDS_7}La sesta parola è: {answer}")

{'vasco': 2, 'rossi': 1, 'gama': 1}

['bollicine', 'da Gama']La sesta parola è: vasco


### BABELNET

In [214]:
def get_indexes_by_word(word):
    service_url = 'https://babelnet.io/v6/getSynsetIds'

    lemma = word
    lang = 'IT'
    key  = 'd4bc0872-d933-4516-b456-2063fa7d68ff'

    params = {
            'lemma' : lemma,
            'searchLang' : lang,
            'key'  : key
    }

    url = service_url + '?' + up.urlencode(params)
    request = urlo.Request(url)
    request.add_header('Accept-encoding', 'gzip')
    response = urlo.urlopen(request)

    if response.info().get('Content-Encoding') == 'gzip':
            buf = BytesIO( response.read())
            f = gzip.GzipFile(fileobj=buf)
            data = json.loads(f.read())
            for result in data:
                return result['id']

In [215]:
def get_synset_by_id(id):
    service_url = 'https://babelnet.io/v6/getSynset'

    key  = 'd4bc0872-d933-4516-b456-2063fa7d68ff'

    params = {
        'id' : id,
        'key'  : key,
        'targetLang' : 'IT'
    }

    url = service_url + '?' + up.urlencode(params)

    request = urlo.Request(url)
    request.add_header('Accept-encoding', 'gzip')
    response = urlo.urlopen(request)
    
    if response.info().get('Content-Encoding') == 'gzip':
        buf = BytesIO(response.read())
        f = gzip.GzipFile(fileobj=buf)
        data = json.loads(f.read())

        # retrieving BabelSense data
        senses = data['senses']
        for result in senses:
            lemma = result.get('lemma')
            language = result.get('language')
            if language != None:    
                print(str(language.encode('utf-8')) + "\t" + str(lemma.encode('utf-8')))

        print('\n')
        # retrieving BabelGloss data
        glosses = data['glosses']
        for result in glosses:
            gloss = result.get('gloss')
            language = result.get('language')
            if language != None:
                print(str(language.encode('utf-8')) + "\t" + str(gloss.encode('utf-8')))

In [216]:
index = get_indexes_by_word("scimmia")
print(index)
if index != []:
    get_synset_by_id(index)

bn:00004847n


b'IT'	b"Gli ominoidi sono una superfamiglia di primati comprendente l'uomo e le cosiddette scimmie antropomorfe, pi\xc3\xb9 simili all'uomo per caratteristiche fisiche e intellettive e comprendenti gli oranghi, gli scimpanz\xc3\xa9, i gorilla e i gibboni."
b'IT'	b'Superfamiglia di primati'
